In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [2]:

alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
base_origin = [0.0, 1.0, 0, 0, 0, 0]
kinematic_dict, K, P, L, D, C, g, qdd, joint_torque, sys_id_coeff = alpha.build_model(root, tip, floating_base=True)
c_parms, m_params, I_params, vec_g, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']
n_joints = kinematic_dict['n_joints']
N = alpha.get_N
D_dot_2C = alpha.get_D_dot_2C
H = alpha.get_total_energy
H_dot = alpha.get_total_power

10


In [3]:
L.size(), K.size(), P.size(), D.size(), g.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1))

In [4]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               q_dot,
                               base_pose,
                               world_pose ],[K, D , C, g, D_dot_2C, N, P])

In [5]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])
c_sample = [5e-3 ,-1e-3 ,16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3]
m_sample = [0.341, 0.341, 0.341, 0.341]
I_sample = [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0]
g_vec_sample = [0, 0, -9.81]
qdot_sample = [0.1, 0.2, 0.3, 0.4]
qdotdot_sample = [0.0, 0.0, 0.0, 0.0]
kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                        m_sample,
                                                                                                        I_sample,
                                                                                                        g_vec_sample,
    q_sample,
    qdot_sample,
    base_T0,
    base_origin
    )

kinetic_energy, robot_inertia_matrix, coriolis_matrix, potential_energy, gravity_vector

(DM(0.00179993),
 DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(
 [[-0.000444861, -0.00247601, 0.0021509, 0.000507749], 
  [0.00250017, 0.000790549, -0.000268627, 0.000441063], 
  [-0.00217081, -0.000524403, 2.48093e-06, -0.000533966], 
  [0.000489242, -0.000508487, 0.000500234, 1.78351e-18]]),
 DM(2.51301),
 DM([-0.0001864, 0.727638, 0.201964, 0.0024038]))

In [6]:
mrc_lump, I_lump  = sys_id_coeff['first_moments'],  sys_id_coeff['inertias_vec6']


mrc_I_i_fun = ca.Function('mrc_I_i', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               q,
                               base_pose,
                               world_pose ], [ca.vertcat(*mrc_lump), ca.vertcat(*I_lump)])

            
m_id_vec = sys_id_coeff['masses_id_syms_vertcat']
m_rci_id_vec = sys_id_coeff['first_moments_id_vertcat']
I_id_vec = sys_id_coeff['inertias_id_vertcat']
 
# alpha.id_theta
# self.id_D, self.id_C, self.id_K, self.id_P, self.id_g, self.Y, self.id_theta
reg_fun = ca.Function('reg', [m_id_vec, m_rci_id_vec, I_id_vec, vec_g, q,q_dot,base_pose,world_pose],[alpha.id_K, alpha.id_D, alpha.id_C, alpha.id_P, alpha.id_g])

mrc_lump_sample , I_lump_sample =  mrc_I_i_fun(c_sample, m_sample, I_sample ,q_sample, base_T0, base_origin)
id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector = reg_fun(m_sample, mrc_lump_sample, I_lump_sample, g_vec_sample, q_sample, qdot_sample, base_T0, base_origin)

id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector 

(DM(0.00179993),
 DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(
 [[-0.000444861, -0.00247601, 0.0021509, 0.000507749], 
  [0.00250017, 0.000790549, -0.000268627, 0.000441063], 
  [-0.00217081, -0.000524403, 2.48093e-06, -0.000533966], 
  [0.000489242, -0.000508487, 0.000500234, 7.34547e-19]]),
 DM(2.51301),
 DM([-0.0001864, 0.727638, 0.201964, 0.0024038]))

In [7]:
tol = 1e-10
errs = {
    "K": float(ca.norm_inf(kinetic_energy - id_kinetic_energy)),
    "D": float(ca.norm_inf(robot_inertia_matrix - id_robot_inertia_matrix)),
    "C": float(ca.norm_inf(coriolis_matrix - id_coriolis_matrix)),
    "P": float(ca.norm_inf(potential_energy - id_potential_energy)),
    "g": float(ca.norm_inf(gravity_vector - id_gravity_vector)),
}
print(errs, {k: (v < tol) for k, v in errs.items()})


{'K': 2.168404344971009e-19, 'D': 1.237345729349082e-17, 'C': 2.168404344971009e-17, 'P': 4.440892098500626e-16, 'g': 1.7763568394002505e-15} {'K': True, 'D': True, 'C': True, 'P': True, 'g': True}


In [8]:
SYS_ID_FUN = ca.Function('SYS_ID_TAU', [m_id_vec, m_rci_id_vec, I_id_vec, vec_g, q,q_dot, q_dotdot, base_pose,world_pose],[alpha.Y@alpha.id_theta])

SYS_ID_FUN(m_sample, mrc_lump_sample, I_lump_sample, g_vec_sample, q_sample, qdot_sample, qdotdot_sample, base_T0, base_origin)

DM([0.000122279, 0.728142, 0.20143, 0.0025011])

In [9]:
Inverse_ID_FUN =ca.Function('Inverse_dynamics_tau', [ca.vertcat(*c_parms), ca.vertcat(*m_params), ca.vertcat(*I_params), vec_g, q, q_dot, q_dotdot, base_pose, world_pose], [alpha.get_inverse_dynamics])
Inverse_ID_FUN(c_sample,m_sample,I_sample,g_vec_sample,q_sample, qdot_sample, qdotdot_sample, base_T0, base_origin)

DM([0.000122279, 0.728142, 0.20143, 0.0025011])

In [10]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                            m_sample,
                                                                                                            I_sample,
                                                                                                            g_vec_sample,
        q_sample,
        qdot_sample,
        base_T0,
        base_origin
        )

    checks = part_properties.is_spd_strict(robot_inertia_matrix) and part_properties.is_skew_symmetric(N_matrix) and np.allclose(D_dot_2C_matrix, N_matrix)
    if not checks:
        raise ValueError(f"check failed for q:{q_sample}")
